In [1]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [2]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
pip install torch


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
# Define a function to create embeddings for each tweet
def create_avg_embeddings(tweet):
    # Tokenize the tweet and convert tokens to IDs
    inputs = tokenizer.encode_plus(tweet, add_special_tokens=True, return_tensors='pt')

    # Pass the input IDs through the model to get embeddings
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()

    # Convert embeddings to a numpy array and return it
    return embeddings.mean(dim=0).numpy()

In [12]:
import os
import torch

In [13]:
# Define the path to the folder containing the text files
data_folder = 'data/'

# Loop through each text file in the data folder and create average embeddings for it
for file_name in os.listdir(data_folder):
    file_path = os.path.join(data_folder, file_name)
    if not os.path.isfile(file_path):
        continue

    # Create average embeddings for the current text file
    avg_embeddings = create_avg_embeddings(file_path)
    print(avg_embeddings)

-0.014243023
-0.013441916
-0.014862657


In [50]:
import csv
import pandas as pd

In [49]:
import os

# Loop through all the CSV files in the directory
directory = '../../go/files/'
output_directory = 'embeddings'  # Output directory for saving the files

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # Read in the tweets from the file
        filepath = os.path.join(directory, filename)
        # Create a DataFrame from the CSV file
        df = pd.read_csv(filepath)
        # Create embeddings for each tweet and store in a list
        embeddings = [create_avg_embeddings(tweet) for tweet in df['Tweet']]
        df['embedding'] = embeddings
        
        # Save the DataFrame with the new column as a new CSV file
        output_filename = 'embeddings_' + filename
        output_filepath = os.path.join(output_directory, output_filename)
        df.to_csv(output_filepath, index=False)